In [ ]:
import numpy as np
import tensorflow as tf
import tflearn
import random
import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
import json
import pickle
import time
import warnings
warnings.filterwarnings("ignore")

stemmer = LancasterStemmer()

In [ ]:
print("Processing intents.....")
with open('intent.json') as json_data:
    intents = json.load(json_data)

In [ ]:
words = []
classes = []
documents = []
ignore_words = ['?']

print("Looping through intents to convert into words, documents, classes and ignore words...")
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word
        word = nltk.word_tokenize(pattern)
        words.extend(word)
        # add documents in the corpus
        documents.append((word, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [ ]:
print("Stemming, lowering and removing duplicates.....")
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# removing the duplicates
classes = sorted(list(set(classes)))

print(len(documents), "documents")
print(len(classes), "classes", classes)
print(len(words), "unique stemmed words", words)

In [ ]:
print("Creating data for the model.....")
training = []
output  = []
print("Creating an empty list for output....")
output_empty = [0] * len(classes)

print("Creating training set, bag of words for each sentence.....")
for doc in documents:
    bag = []
    pattern_words = doc[0]
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]

    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

In [ ]:
print("Shuffling and converting into array for faster processing.....")
random.shuffle(training)
training = np.array(training)

print("Creating train and test lists.....")
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Building neural network for chatbot to be contextual.....")
time.sleep(2)
print("Resetting the graph data.....")
time.sleep(2)
tf.reset_default_graph()

In [ ]:
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)
print("Training the model.....")

In [ ]:
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs') # declaring the model

In [ ]:
print("Training the model.....")
model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)
print("Saving the model.....")
model.save(model.tflearn)

In [ ]:
print("Saving the pickle model.....")
data = pickle.dump({'words': words, 'classes': classes, 'train_x': train_x, 'train_y': train_y}, open("training_data", "wb"))

In [ ]:
print("Loading model.....")
data = pickle.load(open("training_data", "rb"))
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']

with open('intent.json') as json_data:
    intents = json.load(json_data)

print("Loading the model.....")
model.load('./model.tflearn')

In [ ]:
def clean_up_sentence(sentence):
    # It Tokenize or Break it into the constituents parts of Sentense.
    sentence_words = nltk.word_tokenize(sentence)
    # Stemming means to find the root of the word.
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# Return the Array of Bag of Words: True or False and 0 or 1 for each word of bag that exists in the Sentence
def bow(sentence, words, show_details=False):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

ERROR_THRESHOLD = 0.25
print("ERROR_THRESHOLD = 0.25")

def classify(sentence):
    # Prediction or To Get the Posibility or Probability from the Model
    results = model.predict([bow(sentence, words)])[0]
    # Exclude those results which are Below Threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # Sorting is Done because heigher Confidence Answer comes first.
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1])) #Tuppl -> Intent and Probability
    return return_list

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    # That Means if Classification is Done then Find the Matching Tag.
    if results:
        # Long Loop to get the Result.
        while results:
            for i in intents['intents']:
                # Tag Finding
                if i['tag'] == results[0][0]:
                    # Random Response from High Order Probabilities
                    return print(random.choice(i['responses']))

            results.pop(0)

In [ ]:
while True:
    input_data = input("You: ")
    answer = response(input_data)
    answer